In [1]:
import sys
sys.path.append("../")

import jax
import os
os.environ["D4RL_SUPPRESS_IMPORT_ERROR"] = "1"

import warnings
warnings.filterwarnings("ignore")


In [2]:
from icvf_envs.antmaze import d4rl_utils, d4rl_ant, d4rl_pm
from src.gc_dataset import GCSDataset

gcdataset_config = GCSDataset.get_default_config()

env = d4rl_utils.make_env("antmaze-large-diverse-v2")
dataset = d4rl_utils.get_dataset(env)
gc_dataset = GCSDataset(dataset, **gcdataset_config.to_dict())
example_batch = gc_dataset.sample(1)

pybullet build time: May 20 2022 19:45:31
/home/m_bobrin/anaconda3/envs/icvf/lib/python3.9/site-packages/flax/linen/linear.py:30: DeprecationWarning: jax.ShapedArray is deprecated. Use jax.core.ShapedArray
  from jax import ShapedArray


Target Goal:  (32.34774396295872, 25.034585130608814)


load datafile: 100%|██████████| 8/8 [00:02<00:00,  3.24it/s]


In [3]:
import jax.numpy as jnp
import numpy as np

terminal_indxes = np.argwhere(dataset["dones_float"] > 0.5).squeeze()
random_idx = np.random.choice(terminal_indxes, size=2, replace=False)
len(random_idx)
#dataset.get_subset(random_idx)

2

In [6]:
gc_dataset.sample_trajectories(2)[1]['observations']

(10, 29)

In [3]:
example_batch.keys()

dict_keys(['actions', 'dones_float', 'masks', 'next_observations', 'observations', 'rewards', 'desired_rewards', 'desired_masks', 'goals', 'desired_goals'])

#### Each token - (observations, next_observations, dones_float)
For ant_maze obs space is vector of size 29, action - 8

### VQVAE

In [4]:
import optax

import flax.linen as nn
import einops
import jax.numpy as jnp


In [5]:
from dataclasses import dataclass, field

@dataclass
class VqvaeConfig:
    run_opt: str = field(default='train')
    
    adam_beta1: float = .9
    adam_beta2: float = .9
    lr: float = 3e-5
    ema_rate: float = 0.
    n_batch: int = 32   
    warmup_iters: float = 100.
    wd: float = 0.
    grad_clip: float = 200.     
    dtype: str = "float32"
    checkpoint: bool = False
    
    iters_per_ckpt: int = 25000
    iters_per_images: int = 10000
    iters_per_print: int = 1000
    iters_per_save: int = 10000

In [ ]:
from typing import Any


class VQVAE(nn.Module):
    hyperparams: VqvaeConfig
    
    
    @nn.compact
    def __call__(self, *args, **kwargs) -> Any:
        pos_embedding = self.param('pos_embedding', nn.initializers.zeros, [1, num_patches + 1, self.dim])
        pass
    
    def encode(self):
        pass
    
    def decode(self):
        pass